# Running the prompts against several models for code quality comparison

Models:
- GPT 4.1
- GPT 4o
- GPT o3
- GPT o1
- gpt temperature defaults to 1, more creative (range 0 to 2)
- Claude Sonnet 3.7
- Claude Sonnet 4.0
- temperature defaults to 1, more creative
- Deepseek Chat (DeepSeek-V3.1 at time of writing) default temperature 1 (https://api-docs.deepseek.com/api/create-chat-completion)

Keeping default temperature to simulate consumer usage in chat apps like ChatGPT or Anthropics interface
Adding "Only output one alternative to solve this." in order to prevent the model to output multiple suggestions which are harder to evaluate.

Running each prompt 3 times against each model, because:
"Our results reveal that ChatGPT exhibits a high degree
of non-determinism under the default setting: the ratio of coding tasks with zero equal test output across
different requests is 75.76%, 51.00%, and 47.56% for three different code generation datasets" (Ouyang)

Annotating the prompts by characteristics:
- Personification score
- request type (question vs command)
- length?

Some prompts dont elicit the model to write code (eg. RAG tutorial)

In [1]:
import sqlite3
import pandas as pd
import os
from openai import OpenAI
from anthropic import Anthropic
import dotenv

dotenv.load_dotenv()

db_path = "../../giicg.db"
if not os.path.exists(db_path):
    raise FileNotFoundError(f"Database file does not exist: {db_path}")

conn = sqlite3.connect(db_path)
prompts = pd.read_sql("SELECT * FROM translated_scratch_prompts", conn)
prompts = prompts.drop(columns=['conversational', 'code', 'other'])
prompts

,conversation_id,message_id,role,message_text,gender,user_id,programming_language,language
0,6,5,user,I want to use Dummy Hot encoding to replace th...,Woman (cisgender),16,python,en
1,7,43,user,whats the best way to encode and compress a ja...,Man (cisgender),25,javascript,en
2,8,47,user,I have a pandas dataframe like this:\ndata\tpe...,Woman (cisgender),28,python,en
3,10,57,user,"as a NLP and LLM researcher, I am recently dow...",Non-binary,30,python,en
4,12,65,user,Blender and Python. I have a collection of hun...,Man (cisgender),34,python,en
5,13,126,user,"how to run a Python future without blocking, i...",Man (cisgender),46,python,en
6,15,242,user,hey can you write me a short python script for...,Woman (cisgender),48,python,en
7,18,266,user,how can I combine two grib files in jupyter no...,Woman (cisgender),60,python,de
8,20,268,user,I work with Python and have to read an NMEA fi...,Woman (cisgender),65,python,de
9,21,290,user,please write method to unzip file in python,Woman (cisgender),73,python,en


## Models and CLients

In [4]:
gpt_client = OpenAI()
#"o3-2025-04-16", "gpt-5-chat-latest", "gpt-4.1-2025-04-14", "gpt-4o-2024-08-06"
gpt_models = ["chatgpt-4o-latest", "o3-2025-04-16", "gpt-5-chat-latest", "gpt-4.1-2025-04-14"]

deepseek_client = OpenAI(api_key=os.environ.get("DEEPSEEK_API_KEY"), base_url="https://api.deepseek.com")
deepseek_models = ["deepseek-chat"]

anthropic_models = [ "claude-3-7-sonnet-20250219", "claude-sonnet-4-20250514"]
claude_client = Anthropic(
    api_key=os.environ.get("ANTHROPIC_API_KEY"),
)


def prompt_gpt(client, model, prompt):
    response = client.responses.create(
        model=model,
        input=f"{prompt}.Only output one alternative to solve this.",
    )
    return response.output_text


def prompt_deepseek(client, model, prompt):
    response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "user", "content": f"{prompt}. Only output one alternative to solve this.",}
            ],
            stream=False
            #max_tokens=1000,
            #temperature=0.0
        )

    return response.choices[0].message.content


def prompt_claude(client, model, prompt):
    response = client.messages.create(
    messages=[
        {
            "role": "user",
            "content": f"{prompt}. Only output one alternative to solve this.",
        }
    ],
    model=model,
    #temperature=0.0  #Note that even with temperature of 0.0, the results will not be fully deterministic.
    )
    return "".join(block.text for block in response.content if hasattr(block, "text"))

In [14]:
#for model in gpt_models:
 #   prompts[model] = prompts["message_text"].apply(lambda x: prompt_gpt(gpt_client, model, x))

# for model in deepseek_models:
#     prompts[model] = prompts["message_text"].apply(lambda x: prompt_deepseek(deepseek_client, model, x))
#
# for model in anthropic_models:
#     prompts[model] = prompts["message_text"].apply(lambda x: prompt_claude(claude_client, model, x))

In [12]:
# Run each prompt against each model 3 times (instead of once)
import time

runs = 3
prompt_col = "message_text"  # column in `prompts` containing the prompt text
rate_limit_pause = 0.5  # seconds pause between calls to be gentle with APIs


def safe_call(func, client, model, prompt):
    try:
        return func(client, model, prompt)
    except Exception as e:
        # store a small error marker so failures can be inspected later
        return f"__ERROR__: {type(e).__name__}: {e}"


# GPT-family models
# for model in gpt_models:
#     for r in range(1, runs + 1):
#         col = f"{model}_r{r}"
#         print(f"Calling GPT model {model} run {r} -> storing in column {col}")
#         prompts[col] = prompts[prompt_col].apply(lambda x, m=model: safe_call(prompt_gpt, gpt_client, m, x))
#         time.sleep(rate_limit_pause)

# Deepseek models
# for model in deepseek_models:
#     for r in range(1, runs + 1):
#         col = f"{model}_r{r}"
#         print(f"Calling Deepseek model {model} run {r} -> storing in column {col}")
#         prompts[col] = prompts[prompt_col].apply(lambda x, m=model: safe_call(prompt_deepseek, deepseek_client, m, x))
#         time.sleep(rate_limit_pause)

# Claude / Anthropic models
for model in anthropic_models:
    for r in range(1, runs + 1):
        col = f"{model}_r{r}"
        print(f"Calling Anthropic model {model} run {r} -> storing in column {col}")
        prompts[col] = prompts[prompt_col].apply(lambda x, m=model: safe_call(prompt_claude, claude_client, m, x))
        time.sleep(rate_limit_pause)

# Persist the updated dataframe to disk or database as needed; for quick inspection:
prompts

Calling Anthropic model claude-3-7-sonnet-20250219 run 1 -> storing in column claude-3-7-sonnet-20250219_r1
Calling Anthropic model claude-3-7-sonnet-20250219 run 2 -> storing in column claude-3-7-sonnet-20250219_r2
Calling Anthropic model claude-3-7-sonnet-20250219 run 3 -> storing in column claude-3-7-sonnet-20250219_r3
Calling Anthropic model claude-sonnet-4-20250514 run 1 -> storing in column claude-sonnet-4-20250514_r1
Calling Anthropic model claude-sonnet-4-20250514 run 2 -> storing in column claude-sonnet-4-20250514_r2
Calling Anthropic model claude-sonnet-4-20250514 run 3 -> storing in column claude-sonnet-4-20250514_r3


,conversation_id,message_id,role,message_text,gender,user_id,programming_language,language,chatgpt-4o-latest_r1,chatgpt-4o-latest_r2,...,code_gpt-4.1-2025-04-14_r3,code_claude-3-7-sonnet-20250219_r1,code_claude-3-7-sonnet-20250219_r2,code_claude-3-7-sonnet-20250219_r3,code_claude-sonnet-4-20250514_r1,code_claude-sonnet-4-20250514_r2,code_claude-sonnet-4-20250514_r3,code_deepseek-chat_r1,code_deepseek-chat_r2,code_deepseek-chat_r3
0,6,5,user,I want to use Dummy Hot encoding to replace th...,Woman (cisgender),16,python,en,You can use `OneHotEncoder` from `sklearn.prep...,You can use `OneHotEncoder` from `sklearn.prep...,...,from sklearn.preprocessing import OneHotEncode...,from sklearn.preprocessing import OneHotEncode...,from sklearn.preprocessing import OneHotEncode...,from sklearn.preprocessing import OneHotEncode...,from sklearn.preprocessing import OneHotEncode...,,,from sklearn.preprocessing import OneHotEncode...,from sklearn.preprocessing import OneHotEncode...,from sklearn.preprocessing import OneHotEncode...
1,7,43,user,whats the best way to encode and compress a ja...,Man (cisgender),25,javascript,en,Use `JSON.stringify()` to serialize the object...,Use **Base64 encoding of JSON string + URI Com...,...,// Use lz-string for compression\n// Install: ...,function encodeObjectForUrl(obj) {\n // Conve...,function encodeObjectForUrl(obj) {\n // Step ...,function encodeObjectForUrl(obj) {\n // Conve...,// Encode\nfunction encodeObject(obj) {\n ret...,,,function encodeObject(obj) {\n // Convert obj...,function encodeObject(obj) {\n // Convert obj...,function encodeObject(obj) {\n // Convert obj...
2,8,47,user,I have a pandas dataframe like this:\ndata\tpe...,Woman (cisgender),28,python,en,You can use `str.split()` along with `str.extr...,Certainly! Here's one straightforward way to e...,...,import pandas as pd\n\n# Sample DataFrame - re...,import pandas as pd\n\n# Function to parse the...,# Step 1: Split the persona text into key-valu...,import pandas as pd\n\n# Function to parse the...,import pandas as pd\n\ndef parse_persona(perso...,,,# Split the persona column on newlines and exp...,# Split the persona column by newlines and exp...,# Split the persona column by newlines and exp...
3,10,57,user,"as a NLP and LLM researcher, I am recently dow...",Non-binary,30,python,en,Your idea is **highly practical and implementa...,Your idea is both **practical** and **implemen...,...,import os\nfrom pypdf import PdfReader\nfrom p...,import os\nimport re\nfrom typing import Optio...,import os\nimport re\nfrom pathlib import Path...,import os\nimport re\nimport PyPDF2\nfrom path...,def extract_title(pdf_path):\n # Try metada...,,,import os\nimport re\nimport PyPDF2\nimport re...,import os\nimport re\nimport fitz # PyMuPDF\n...,import fitz # PyMuPDF\n\n def extract_text...
4,12,65,user,Blender and Python. I have a collection of hun...,Man (cisgender),34,python,en,Here is a **Blender Python script** that creat...,Certainly! Below is a **Blender Python script*...,...,import bpy\nimport mathutils\n\n# CONFIGURATIO...,import bpy\nimport mathutils\nimport math\n\nd...,,,import bpy\nimport bmesh\nfrom mathutils impor...,,,import bpy\nimport bmesh\nfrom mathutils impor...,import bpy\nimport bmesh\nfrom mathutils impor...,import bpy\nimport bmesh\nfrom mathutils impor...
5,13,126,user,"how to run a Python future without blocking, i...",Man (cisgender),46,python,en,You can use `task = asyncio.create_task(coro)`...,You can use `asyncio.Task` and `task.step()` i...,...,import asyncio\n\nasync def my_coro():\n pr...,import asyncio\nfrom functools import wraps\ni...,import asyncio\nfrom functools import partial\...,import asyncio\nfrom functools import partial\...,import asyncio\n\nasync def my_coroutine():\n ...,,,import asyncio\n\nasync def my_coroutine():\n ...,import asyncio\n\nasync def my_async_function(...,import asyncio\n\nasync def my_coroutine():\n ...
6,15,242,user,hey can you write me a short python script for...,Woman (cisgender),48,pytho

## Parse Code Snippets

In [13]:
import re

def parse_code_snippets(text):
    """
    Extract code snippets marked with triple backticks and any of these language tags:
    python, javascript, js, typescript, ts.
    Joins all found code into a single string separated by newlines.
    """
    pattern = r"```(?:python|javascript|js|typescript|ts|html)[ \t]*\n(.*?)(?=```)"
    matches = re.findall(pattern, text, re.DOTALL | re.IGNORECASE)
    code_snippets = [m.strip() for m in matches]
    return "\n".join(code_snippets)

def extract_really_all_code_snippets(text):
    """
    Extract code snippets enclosed in triple backticks, optionally with any language tag.
    Joins all found code into a single string separated by newlines.
    Returns an empty string if none found.
    """
    pattern = r"```(?:\w+)?\n(.*?)```"
    matches = re.findall(pattern, text, re.DOTALL)
    code_snippets = [m.strip() for m in matches]
    return "\n".join(code_snippets)



base_columns = ["chatgpt-4o-latest", "o3-2025-04-16", "gpt-5-chat-latest", "gpt-4.1-2025-04-14", "claude-3-7-sonnet-20250219", "claude-sonnet-4-20250514", "deepseek-chat"]

for column in base_columns:
    code_run_cols = []
    for r in range(1, runs + 1):
        src_col = f"{column}_r{r}"
        dest_col = f"code_{column}_r{r}"
        code_run_cols.append(dest_col)
        if src_col in prompts.columns:
            prompts[dest_col] = prompts[src_col].apply(parse_code_snippets)

def infer_source_col(colname):
    """
    Given a result col name like 'code_claude-3-7-sonnet-20250219_r3',
    returns the source col: drops the 'code_' prefix if present.
    Example: 'code_claude-3-7-sonnet-20250219_r3' -> 'claude-3-7-sonnet-20250219_r3'
    """
    return colname[5:] if colname.startswith("code_") else None

for col in prompts.columns:
    source_col = infer_source_col(col)
    if source_col is not None and source_col in prompts.columns:
        # Only fill code_ columns where their value is empty/NA and the source exists
        for row_idx in prompts.index[prompts[col].isna() | (prompts[col] == "")]:
            src_text = prompts.at[row_idx, source_col]
            # If the source itself is empty, keep target empty
            if pd.notna(src_text) and src_text != "":
                code = extract_really_all_code_snippets(src_text)
                prompts.at[row_idx, col] = code

prompts


,conversation_id,message_id,role,message_text,gender,user_id,programming_language,language,chatgpt-4o-latest_r1,chatgpt-4o-latest_r2,...,code_gpt-4.1-2025-04-14_r3,code_claude-3-7-sonnet-20250219_r1,code_claude-3-7-sonnet-20250219_r2,code_claude-3-7-sonnet-20250219_r3,code_claude-sonnet-4-20250514_r1,code_claude-sonnet-4-20250514_r2,code_claude-sonnet-4-20250514_r3,code_deepseek-chat_r1,code_deepseek-chat_r2,code_deepseek-chat_r3
0,6,5,user,I want to use Dummy Hot encoding to replace th...,Woman (cisgender),16,python,en,You can use `OneHotEncoder` from `sklearn.prep...,You can use `OneHotEncoder` from `sklearn.prep...,...,from sklearn.preprocessing import OneHotEncode...,from sklearn.preprocessing import OneHotEncode...,from sklearn.preprocessing import OneHotEncode...,from sklearn.preprocessing import OneHotEncode...,from sklearn.preprocessing import OneHotEncode...,from sklearn.preprocessing import OneHotEncode...,from sklearn.preprocessing import OneHotEncode...,from sklearn.preprocessing import OneHotEncode...,from sklearn.preprocessing import OneHotEncode...,from sklearn.preprocessing import OneHotEncode...
1,7,43,user,whats the best way to encode and compress a ja...,Man (cisgender),25,javascript,en,Use `JSON.stringify()` to serialize the object...,Use **Base64 encoding of JSON string + URI Com...,...,// Use lz-string for compression\n// Install: ...,function compressObjectForUrl(obj) {\n // Con...,function encodeObjectForUrl(obj) {\n // 1. Co...,function encodeObjectForUrl(obj) {\n // Conve...,function encodeObject(obj) {\n return btoa(JS...,// Encode\nfunction encodeObject(obj) {\n ret...,// Encode\nfunction encodeObject(obj) {\n ret...,function encodeObject(obj) {\n // Convert obj...,function encodeObject(obj) {\n // Convert obj...,function encodeObject(obj) {\n // Convert obj...
2,8,47,user,I have a pandas dataframe like this:\ndata\tpe...,Woman (cisgender),28,python,en,You can use `str.split()` along with `str.extr...,Certainly! Here's one straightforward way to e...,...,import pandas as pd\n\n# Sample DataFrame - re...,def parse_persona(persona_text):\n # Split ...,# Convert persona strings into a dictionary\nd...,import pandas as pd\nimport re\n\n# Function t...,# Split the persona column and create new colu...,import pandas as pd\n\n# Split the persona col...,import pandas as pd\n\n# Function to parse per...,# Split the persona column on newlines and exp...,# Split the persona column by newlines and exp...,# Split the persona column by newlines and exp...
3,10,57,user,"as a NLP and LLM researcher, I am recently dow...",Non-binary,30,python,en,Your idea is **highly practical and implementa...,Your idea is both **practical** and **implemen...,...,import os\nfrom pypdf import PdfReader\nfrom p...,import os\nimport re\nimport PyPDF2\nfrom path...,import os\nimport re\nimport glob\nfrom pdfmin...,import os\nimport re\nimport PyPDF2\nimport ar...,import fitz # PyMuPDF\nimport re\nimport os\n...,import PyPDF2 # or pdfplumber for better text...,import os\nimport re\nimport PyPDF2\nfrom path...,import os\nimport re\nimport PyPDF2\nimport re...,import os\nimport re\nimport fitz # PyMuPDF\n...,import fitz # PyMuPDF\n\n def extract_text...
4,12,65,user,Blender and Python. I have a collection of hun...,Man (cisgender),34,python,en,Here is a **Blender Python script** that creat...,Certainly! Below is a **Blender Python script*...,...,import bpy\nimport mathutils\n\n# CONFIGURATIO...,import bpy\nimport mathutils\n\ndef create_cub...,import bpy\nimport mathutils\nfrom math import...,import bpy\nimport mathutils\nfrom math import...,import bpy\nimport bmesh\nfrom mathutils impor...,import bpy\nimport bmesh\nfrom mathutils impor...,import bpy\nimport bmesh\nfrom mathutils impor...,import bpy\nimport bmesh\nfrom mathutils impor...,import bpy\nimport bmesh\nfrom mathutils impor...,import bpy\nimport bmesh\nfrom mathutils impor...
5,13,126,user,"how to run a Python future without blocking, i...",Man (cisgender),46,python,en,You can u

## intermediate save

In [14]:
prompts.to_sql("tmp_prompts_with_code", conn, if_exists="replace", index=False)

34

In [4]:
import sqlite3
import pandas as pd
import os
from openai import OpenAI
from anthropic import Anthropic
import dotenv

dotenv.load_dotenv()
db_path = "../../giicg.db"
if not os.path.exists(db_path):
    raise FileNotFoundError(f"Database file does not exist: {db_path}")

conn = sqlite3.connect(db_path)
prompts = pd.read_sql("SELECT * FROM tmp_prompts_with_code", conn)
prompts

,conversation_id,message_id,role,message_text,gender,user_id,programming_language,language,chatgpt-4o-latest_r1,chatgpt-4o-latest_r2,...,code_gpt-4.1-2025-04-14_r3,code_claude-3-7-sonnet-20250219_r1,code_claude-3-7-sonnet-20250219_r2,code_claude-3-7-sonnet-20250219_r3,code_claude-sonnet-4-20250514_r1,code_claude-sonnet-4-20250514_r2,code_claude-sonnet-4-20250514_r3,code_deepseek-chat_r1,code_deepseek-chat_r2,code_deepseek-chat_r3
0,6,5,user,I want to use Dummy Hot encoding to replace th...,Woman (cisgender),16,python,en,You can use `OneHotEncoder` from `sklearn.prep...,You can use `OneHotEncoder` from `sklearn.prep...,...,from sklearn.preprocessing import OneHotEncode...,from sklearn.preprocessing import OneHotEncode...,from sklearn.preprocessing import OneHotEncode...,from sklearn.preprocessing import OneHotEncode...,from sklearn.preprocessing import OneHotEncode...,from sklearn.preprocessing import OneHotEncode...,from sklearn.preprocessing import OneHotEncode...,from sklearn.preprocessing import OneHotEncode...,from sklearn.preprocessing import OneHotEncode...,from sklearn.preprocessing import OneHotEncode...
1,7,43,user,whats the best way to encode and compress a ja...,Man (cisgender),25,javascript,en,Use `JSON.stringify()` to serialize the object...,Use **Base64 encoding of JSON string + URI Com...,...,// Use lz-string for compression\n// Install: ...,function compressObjectForUrl(obj) {\n // Con...,function encodeObjectForUrl(obj) {\n // 1. Co...,function encodeObjectForUrl(obj) {\n // Conve...,function encodeObject(obj) {\n return btoa(JS...,// Encode\nfunction encodeObject(obj) {\n ret...,// Encode\nfunction encodeObject(obj) {\n ret...,function encodeObject(obj) {\n // Convert obj...,function encodeObject(obj) {\n // Convert obj...,function encodeObject(obj) {\n // Convert obj...
2,8,47,user,I have a pandas dataframe like this:\ndata\tpe...,Woman (cisgender),28,python,en,You can use `str.split()` along with `str.extr...,Certainly! Here's one straightforward way to e...,...,import pandas as pd\n\n# Sample DataFrame - re...,def parse_persona(persona_text):\n # Split ...,# Convert persona strings into a dictionary\nd...,import pandas as pd\nimport re\n\n# Function t...,# Split the persona column and create new colu...,import pandas as pd\n\n# Split the persona col...,import pandas as pd\n\n# Function to parse per...,# Split the persona column on newlines and exp...,# Split the persona column by newlines and exp...,# Split the persona column by newlines and exp...
3,10,57,user,"as a NLP and LLM researcher, I am recently dow...",Non-binary,30,python,en,Your idea is **highly practical and implementa...,Your idea is both **practical** and **implemen...,...,import os\nfrom pypdf import PdfReader\nfrom p...,import os\nimport re\nimport PyPDF2\nfrom path...,import os\nimport re\nimport glob\nfrom pdfmin...,import os\nimport re\nimport PyPDF2\nimport ar...,import fitz # PyMuPDF\nimport re\nimport os\n...,import PyPDF2 # or pdfplumber for better text...,import os\nimport re\nimport PyPDF2\nfrom path...,import os\nimport re\nimport PyPDF2\nimport re...,import os\nimport re\nimport fitz # PyMuPDF\n...,import fitz # PyMuPDF\n\n def extract_text...
4,12,65,user,Blender and Python. I have a collection of hun...,Man (cisgender),34,python,en,Here is a **Blender Python script** that creat...,Certainly! Below is a **Blender Python script*...,...,import bpy\nimport mathutils\n\n# CONFIGURATIO...,import bpy\nimport mathutils\n\ndef create_cub...,import bpy\nimport mathutils\nfrom math import...,import bpy\nimport mathutils\nfrom math import...,import bpy\nimport bmesh\nfrom mathutils impor...,import bpy\nimport bmesh\nfrom mathutils impor...,import bpy\nimport bmesh\nfrom mathutils impor...,import bpy\nimport bmesh\nfrom mathutils impor...,import bpy\nimport bmesh\nfrom mathutils impor...,import bpy\nimport bmesh\nfrom mathutils impor...
5,13,126,user,"how to run a Python future without blocking, i...",Man (cisgender),46,python,en,You can u

## Pivot columns for saving

In [10]:

#model_columns = ["chatgpt-4o-latest", "o3-2025-04-16", "gpt-5-chat-latest", "gpt-4.1-2025-04-14", "claude-3-7-sonnet-20250219", "claude-sonnet-4-20250514"]
# code_columns = [f"code_{col}" for col in columns if f"code_{col}" in prompts.columns]
# print(code_columns)
base_columns = ["chatgpt-4o-latest_r1",
                "chatgpt-4o-latest_r2",
                "chatgpt-4o-latest_r3",

                "o3-2025-04-16_r1",
                "o3-2025-04-16_r2",
                "o3-2025-04-16_r3",

                "gpt-5-chat-latest_r1",
                "gpt-5-chat-latest_r2",
                "gpt-5-chat-latest_r3",

                "gpt-4.1-2025-04-14_r1",
                "gpt-4.1-2025-04-14_r2",
                "gpt-4.1-2025-04-14_r3",

                "claude-3-7-sonnet-20250219_r1",
                "claude-3-7-sonnet-20250219_r2",
                "claude-3-7-sonnet-20250219_r3",

                "claude-sonnet-4-20250514_r1",
                "claude-sonnet-4-20250514_r2",
                "claude-sonnet-4-20250514_r3",

                "deepseek-chat_r1",
                "deepseek-chat_r2",
                "deepseek-chat_r3"]

melted = prompts.melt(
    id_vars=["conversation_id", "message_id", "message_text", "gender", "user_id", "programming_language"],
    value_vars=base_columns,
    var_name="model_version",
    value_name="model_reply"
)

code_columns = [f"code_{col}" for col in base_columns if f"code_{col}" in prompts.columns]

if code_columns:
    code_melted = prompts.melt(
        id_vars=["conversation_id", "message_id", "message_text", "gender", "user_id", "programming_language"],
        value_vars=code_columns,
        var_name="model_version_code",
        value_name="code"
    )

    code_melted["model_version"] = code_melted["model_version_code"].str.replace("code_", "")

    melted = melted.merge(
        code_melted[["conversation_id", "message_id", "model_version", "code"]],
        on=["conversation_id", "message_id", "model_version"],
        how="left"
    )
else:

    melted["code"] = None


melted = melted.reset_index(drop=True)
melted["code_reply_id"] = melted.index + 1


final_df = melted[
    ["code_reply_id", "conversation_id", "message_id", "message_text", "gender", "user_id",
     "model_reply", "code", "model_version", "programming_language"]
]


final_df

,code_reply_id,conversation_id,message_id,message_text,gender,user_id,model_reply,code,model_version,programming_language
0,1,6,5,I want to use Dummy Hot encoding to replace th...,Woman (cisgender),16,You can use `OneHotEncoder` from `sklearn.prep...,from sklearn.preprocessing import OneHotEncode...,chatgpt-4o-latest_r1,python
1,2,7,43,whats the best way to encode and compress a ja...,Man (cisgender),25,Use `JSON.stringify()` to serialize the object...,import LZString from 'lz-string';\n\nfunction ...,chatgpt-4o-latest_r1,javascript
2,3,8,47,I have a pandas dataframe like this:\ndata\tpe...,Woman (cisgender),28,You can use `str.split()` along with `str.extr...,import pandas as pd\n\n# Assuming your DataFra...,chatgpt-4o-latest_r1,python
3,4,10,57,"as a NLP and LLM researcher, I am recently dow...",Non-binary,30,Your idea is **highly practical and implementa...,import os\n import requests\n import time\...,chatgpt-4o-latest_r1,python
4,5,12,65,Blender and Python. I have a collection of hun...,Man (cisgender),34,Here is a **Blender Python script** that creat...,import bpy\nimport mathutils\n\n# === CONFIGUR...,chatgpt-4o-latest_r1,python
...,...,...,...,...,...,...,...,...,...,...
709,710,70,1524,can you write a test that tests whether parse ...,Woman (cisgender),90,Here's a test that verifies `parse_obj` works ...,import pytest\nfrom pydantic import Validation...,deepseek-chat_r3,python
710,711,72,1534,can you write me some python code to count the...,Woman (cisgender),11,Here's a simple Python function to count the n...,import json\n\ndef count_objects_in_json(data)...,deepseek-chat_r3,python
711,712,74,1538,I want to merge these two dataframes: \ndf_tra...,Woman (cisgender),28,You can merge the dataframes using a left merg...,merged_df = df_train_claim.merge(df_train_clai...,deepseek-chat_r3,python
712,713,78,1598,"0,tcp,http,SF,215,45076,0,0,0,0,0,1,0,0,0,0,0,...",Woman (cisgender),73,To determine the best k for k-means clustering...,from sklearn.cluster import KMeans\nfrom sklea...,deepseek-chat_r3,python


In [12]:
final_df["model_version"] = final_df["model_version"].str.replace(r"_r\d+$", "", regex=True)
final_df

,code_reply_id,conversation_id,message_id,message_text,gender,user_id,model_reply,code,model_version,programming_language
0,1,6,5,I want to use Dummy Hot encoding to replace th...,Woman (cisgender),16,You can use `OneHotEncoder` from `sklearn.prep...,from sklearn.preprocessing import OneHotEncode...,chatgpt-4o-latest,python
1,2,7,43,whats the best way to encode and compress a ja...,Man (cisgender),25,Use `JSON.stringify()` to serialize the object...,import LZString from 'lz-string';\n\nfunction ...,chatgpt-4o-latest,javascript
2,3,8,47,I have a pandas dataframe like this:\ndata\tpe...,Woman (cisgender),28,You can use `str.split()` along with `str.extr...,import pandas as pd\n\n# Assuming your DataFra...,chatgpt-4o-latest,python
3,4,10,57,"as a NLP and LLM researcher, I am recently dow...",Non-binary,30,Your idea is **highly practical and implementa...,import os\n import requests\n import time\...,chatgpt-4o-latest,python
4,5,12,65,Blender and Python. I have a collection of hun...,Man (cisgender),34,Here is a **Blender Python script** that creat...,import bpy\nimport mathutils\n\n# === CONFIGUR...,chatgpt-4o-latest,python
...,...,...,...,...,...,...,...,...,...,...
709,710,70,1524,can you write a test that tests whether parse ...,Woman (cisgender),90,Here's a test that verifies `parse_obj` works ...,import pytest\nfrom pydantic import Validation...,deepseek-chat,python
710,711,72,1534,can you write me some python code to count the...,Woman (cisgender),11,Here's a simple Python function to count the n...,import json\n\ndef count_objects_in_json(data)...,deepseek-chat,python
711,712,74,1538,I want to merge these two dataframes: \ndf_tra...,Woman (cisgender),28,You can merge the dataframes using a left merg...,merged_df = df_train_claim.merge(df_train_clai...,deepseek-chat,python
712,713,78,1598,"0,tcp,http,SF,215,45076,0,0,0,0,0,1,0,0,0,0,0,...",Woman (cisgender),73,To determine the best k for k-means clustering...,from sklearn.cluster import KMeans\nfrom sklea...,deepseek-chat,python


## Save to database

In [13]:
∞final_df.to_sql("translated_scratch_prompts_with_code", conn, if_exists="replace", index=False)

714